<a href="https://colab.research.google.com/github/jwb4335/nlp/blob/master/hw_part1/hw1_part1_johnbarry_20200125.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HW1 Part 1
John Barry
COMPSCI 590.07


New topics in SLP3 are:
- Neural networks and neural language models
- Why: Neural networks have pushed to the forefront of machine learning and natural language processing. They have to be there
- Sequence processing with recurrent networks
- Why: same reason
- Encoder-decoder models and contextual embeddings
- Why: Another example neural networks
- Constituency grammar
- Why: It is natural to think of language in grammatical units as opposed to words. This is probably not new but just an update of the syntax section in SLP2. 

Retired topics from SLP2 are:
- Hidden Markov Models (actually moved to appendix)
- Why: Been overtaken by neural networks
- Phonetics and Speech synthesis(actually moved down a lot)
- Why: I don't know, this is more to do with speech I guess. Maybe machines are bad at phonetic parsing
- Computational phonology
- Why: Maybe because it's less interesting.
- Lexical semantics:
- Why: Lexical semantics focuses on breaking down words into lexical units. As machine learning in NLP relies on context, it makes more sense to build up into grammatical or semantical units of collocoated words. I guess?


In [119]:
#!pip3 install --quiet nltk 
#!pip3 install --quiet textract

import os
import pickle
import textract
import nltk
import urllib
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
from nltk import word_tokenize
from nltk.util import ngrams
import math
from collections import Counter

## Load in the SLP2 and SLP3 toc's
## NOTE: The blotted-out code converts the PDFs to text. The urls are github links to those textfiles
"""
slp2 = textract.process('Table of Contents slp-2.pdf').decode('utf-8')
slp3 = textract.process('Table of Contents slp-3.pdf').decode('utf-8')
slp2 = '#' + slp2
spl3 = '#' + slp3
with open('slp2.txt', 'w') as f:
    f.write(slp2)
with open('slp3.txt', 'w') as f:
    f.write(slp3)
"""
url2 = 'https://raw.githubusercontent.com/jwb4335/nlp/master/hw1_part1/slp2.txt'
url3 = 'https://raw.githubusercontent.com/jwb4335/nlp/master/hw1_part1/slp3.txt'

slp2 = urllib.request.urlopen(url2).read().decode('utf-8')
slp3 = urllib.request.urlopen(url3).read().decode('utf-8')

## Store the objects for the loop
contents = [slp2,slp3]

## Create a store matrix for the bigrams
bigrams = [[''],['']]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [120]:
## We need to clean the toc's a little. Remove from non-interesting words, numbers and symbols
i = 0
for toc in contents:
    toc = toc.replace('. ','')
    toc = ''.join([i for i in toc if not i.isdigit()])
    toc = toc.replace('Summary of Contents','')
    toc = toc.replace('Preface .xxiii','')
    toc = toc.replace('Introduction','')
    toc = toc.replace('Bibliography','')
    toc = toc.replace('Index','')
    toc = toc.replace('vii','')
    toc = toc.replace('I Words','')
    toc = toc.replace('II Speech','')
    toc = toc.replace('III Syntax','')
    toc = toc.replace('IV Semantics and Pragmatics','')
    toc = toc.replace('Applications','')
    toc = toc.replace('\nV','')
    toc = toc.replace('\nD','',1)
    toc = toc.replace('\nRA','')
    toc = toc.replace('\nFT','')
    toc = toc.replace('&','and')
    toc = toc.replace(':',' ')
    toc = toc.replace(',',' ')   
    toc = toc.replace('#','')
    token = word_tokenize(toc)
    bigrams[i] = list(ngrams(token,2))
    i = i+1
""" 
print('The bigram for SLP2:')
for line in bigrams[0]:
    print(line)
    
print('\n\n')
print('The bigram for SLP3:')
for line in bigrams[1]:
    print(line)
"""

" \nprint('The bigram for SLP2:')\nfor line in bigrams[0]:\n    print(line)\n    \nprint('\n\n')\nprint('The bigram for SLP3:')\nfor line in bigrams[1]:\n    print(line)\n"

In [132]:
"""def ngram_cos_sim(ngram_1,ngram_2):
  # Get the n-gram counts of each vector
  vec_1_dict = dict(Counter(ngram_1))
  vec_2_dict = dict(Counter(ngram_2))
  a = np.fromiter(.values(),dtype = 'float')
  b = np.fromiter(.values(),dtype = 'float')"""
ngram_1 = sorted(bigrams[0])
ngram_2 = sorted(bigrams[1])
ngram_all = sorted(list(set(ngram_1 + ngram_2)))
vec_1 = Counter(ngram_1)
vec_2 = Counter(ngram_2)
print('Here are the bigrams that appear in both TOCs:\n{}'.format(ngram_all))

Here are the bigrams that appear in both TOCs:
[('A', 'Hidden'), ('Advanced', 'Topics'), ('Affect', 'and'), ('Agents', 'Machine'), ('Answering', 'Dialogue'), ('Answering', 'and'), ('Appendix', 'A'), ('Attention', 'and'), ('Automata', 'Words'), ('Automatic', 'Speech'), ('Bayes', 'and'), ('Chatbots', 'Phonetics'), ('Classification', 'Logistic'), ('Coherence', 'Summarization'), ('Complexity', 'Representing'), ('Computational', 'Discourse'), ('Computational', 'Lexical'), ('Computational', 'Phonology'), ('Computational', 'Semantics'), ('Connotation', 'Coreference'), ('Constituency', 'Grammars'), ('Constituency', 'Parsing'), ('Context-Free', 'Grammars'), ('Contextual', 'Embeddings'), ('Conversational', 'Agents'), ('Coreference', 'Resolution'), ('Dialogue', 'Systems'), ('Dialogue', 'and'), ('Discourse', 'Coherence'), ('Discourse', 'Information'), ('Distance', 'N-gram'), ('Edit', 'Distance'), ('Embeddings', 'Machine'), ('Embeddings', 'Neural'), ('Encoder-Decoder', 'Models'), ('English', 'Parsi

In [134]:
## This defines the cosine similarity measure. 
def cos_sim_count(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

cos_sim = round(cos_sim_count(vec_1,vec_2),4)

print("""The cosine similarity is {}. This is lower than the score I would give, it does show that the TOCs have changed a lot""".format(cos_sim))

The cosine similarity is 0.129. This is lower than the score I would give, it does show that the TOCs have changed a lot
